In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

True
1
NVIDIA RTX A6000


In [3]:

from torch import nn
import numpy as np
import glob
import os
from PIL import Image
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
import torch
from tqdm import tqdm
from collections import namedtuple
import copy
import random
import shutil
from torchvision import models
from torch.nn import functional as F


In [5]:
"""
RCAN model
"""
class SpectralAttention(nn.Module):
    def __init__(self, num_features, reduction):
        super(SpectralAttention, self).__init__()
        self.module = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(num_features, num_features // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features // reduction, num_features, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.module(x)

class ChannelAttention(nn.Module):
    def __init__(self, num_features, reduction):
        super(ChannelAttention, self).__init__()
        self.module = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(num_features, num_features // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features // reduction, num_features, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.module(x)

class RCAB(nn.Module):
    def __init__(self, num_features, reduction):
        super(RCAB, self).__init__()
        self.module = nn.Sequential(
            nn.Conv2d(num_features, num_features, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features, num_features, kernel_size=3, padding=1),
            ChannelAttention(num_features, reduction)
        )

    def forward(self, x):
        return x + self.module(x)

class RG(nn.Module):
    def __init__(self, num_features, num_rcab, reduction):
        super(RG, self).__init__()
        self.module = [RCAB(num_features, reduction) for _ in range(num_rcab)]
        self.module.append(nn.Conv2d(num_features, num_features, kernel_size=3, padding=1))
        self.module = nn.Sequential(*self.module)

    def forward(self, x):
        return x + self.module(x)

class RCAN(nn.Module):
    def __init__(self, args):
        super(RCAN, self).__init__()
        scale = args.scale
        num_features = args.num_features
        num_rg = args.num_rg
        num_rcab = args.num_rcab
        reduction = args.reduction

        self.sf = nn.Conv2d(13, num_features, kernel_size=3, padding=1) # modified from 3 to 13
        self.spectral_attention = SpectralAttention(num_features, reduction) #added spectral attention
        # RIR layers
        self.rgs = nn.Sequential(*[RG(num_features, num_rcab, reduction) for _ in range(num_rg)])
        self.conv1 = nn.Conv2d(num_features, num_features, kernel_size=3, padding=1)
        self.upscale = nn.Sequential(
            nn.Conv2d(num_features, num_features * (scale ** 2), kernel_size=3, padding=1),
            nn.PixelShuffle(scale)
        )
        self.conv2 = nn.Conv2d(num_features, 13, kernel_size=3, padding=1) # modified to 13 output channels

    def forward(self, x):
        x = self.sf(x)
        x = self.spectral_attention(x) #applying spectral attention here
        residual = x
        x = self.rgs(x)
        x = self.conv1(x)
        x += residual
        x = self.upscale(x)
        x = self.conv2(x)
        return x

class UNet(nn.Module):
  def __init__(self, in_channels, out_channels, features=64):
        super(UNet, self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv1 = nn.Conv2d(in_channels, features, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.conv2 = nn.Conv2d(features, features*2, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.conv3 = nn.Conv2d(features*2, features*4, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.conv4 = nn.Conv2d(features*4, features*8, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.conv5 = nn.ConvTranspose2d(features*8, features*4, kernel_size=(2, 2), stride=(2,2))
        self.conv6 = nn.ConvTranspose2d(features*8, features*2, kernel_size=(2, 2), stride=(2,2))
        self.conv7 = nn.ConvTranspose2d(features*4, features, kernel_size=(2, 2), stride=(2,2))
        self.conv8 = nn.Conv2d(features*2, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=1)

  def forward(self, x):
     x1 = F.relu(self.conv1(x))
     x2 = F.relu(self.conv2(self.pool(x1)))
     x3 = F.relu(self.conv3(self.pool(x2)))
     x4 = F.relu(self.conv4(self.pool(x3)))
     x5 = F.relu(self.conv5(x4))
     x6 = torch.cat([x5, x3], dim = 1)
     x6 = F.relu(self.conv6(x6))
     x7 = torch.cat([x6, x2], dim = 1)
     x7 = F.relu(self.conv7(x7))
     x8 = torch.cat([x7, x1], dim = 1)
     out = F.relu(self.conv8(x8))
     return out

class VGG19(nn.Module):
    def __init__(self, pretrained=True, end_with='pool4'):
        super().__init__()
        self.features = models.vgg19(pretrained=pretrained).features
        self.end_with = end_with
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        for name, module in self.features._modules.items():
            x = module(x)
            if name == self.end_with:
              return x
        return x

"""
Setup the dataset
"""

def split_dataset(eurosat_path, dataset_path, split_ratio=(0.7, 0.15, 0.15)):
    """Splits the eurosat dataset into train, validation, and test sets."""
    if os.path.exists(dataset_path):
      print("Dataset folder already exists, skipping dataset creation...")
      return
    os.makedirs(dataset_path, exist_ok=True)
    classes = [d for d in os.listdir(eurosat_path) if os.path.isdir(os.path.join(eurosat_path, d))]

    for split_type in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dataset_path, split_type), exist_ok=True)
        for class_name in classes:
             os.makedirs(os.path.join(dataset_path, split_type, class_name), exist_ok=True)

    for class_name in classes:
        class_path = os.path.join(eurosat_path, class_name)
        image_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

        random.shuffle(image_files)
        train_size = int(len(image_files) * split_ratio[0])
        val_size = int(len(image_files) * split_ratio[1])
        # remaining will be test

        train_files = image_files[:train_size]
        val_files = image_files[train_size:train_size + val_size]
        test_files = image_files[train_size + val_size:]

        for split_type, files in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
            for file_name in files:
                src_path = os.path.join(class_path, file_name)
                dst_path = os.path.join(dataset_path, split_type, file_name)
                print(f"Copying {src_path} to {dst_path}")
                shutil.copyfile(src_path, dst_path)

class CustomDataset(Dataset):
    def __init__(self, dataset_path, scale=4):
        super(CustomDataset, self).__init__()
        self.image_paths = []
        for dirpath, dirnames, filenames in os.walk(dataset_path):
            for filename in filenames:
               if filename.endswith('.png') or filename.endswith('.jpg'):
                  self.image_paths.append(os.path.join(dirpath, filename))
        self.scale = scale

    def __getitem__(self, idx):
        hr_path = self.image_paths[idx]
        hr = Image.open(hr_path)
        # resize images
        hr = hr.resize((64,64), resample=Image.BICUBIC)

        lr_size = (64 // self.scale, 64 // self.scale)
        lr = hr.resize(lr_size, resample = Image.BICUBIC) # creates lr from hr
        
        # convert data type
        hr = np.array(hr).astype(np.float32)
        lr = np.array(lr).astype(np.float32)
        # take only the first 13 channels of data
        if hr.shape[2] > 13:
            hr = hr[...,:13]
            lr = lr[...,:13]
        elif hr.shape[2] < 13: # padding case
            channels_to_add = 13 - hr.shape[2]
            hr = np.pad(hr, ((0, 0), (0, 0), (0, channels_to_add)), 'constant')
            lr = np.pad(lr, ((0, 0), (0, 0), (0, channels_to_add)), 'constant')
        # transpose and normalize
        hr = np.transpose(hr, axes=[2, 0, 1])
        lr = np.transpose(lr, axes=[2, 0, 1])
        
        hr /= 255.0
        lr /= 255.0
        return lr, hr

    def __len__(self):
        return len(self.image_paths)

# creates h5 data files for you in the directory of the `path`
eurosat_path = 'archive\eurosat'
dataset_path = 'dataset'
split_dataset(eurosat_path, dataset_path) # splits your data
"""
Model Setup
"""
torch.manual_seed(123)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ParamStrct = namedtuple('ParamStrct', 'scale num_features num_rg num_rcab reduction')
param = ParamStrct(4, 64, 6, 16, 16)
batch_size = 32 # adjust the batch size to your resource and preference

model = RCAN(param).to(device)
unet_model = UNet(in_channels = 13, out_channels = 13, features = 64).to(device)
vgg_model = VGG19(pretrained = True, end_with = 'pool4').to(device)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer_unet = optim.Adam(unet_model.parameters(), lr = 1e-4)
"""
Setup data loader
"""
train_dataset = CustomDataset(os.path.join(dataset_path, 'train'))
train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0,
                                  pin_memory=True,
                                  drop_last=True)
eval_dataset = CustomDataset(os.path.join(dataset_path, 'val'))
eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=batch_size)
test_dataset = CustomDataset(os.path.join(dataset_path, 'test'))
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1) # batch size is 1 here to evaluate each image independently
"""
Util function to measure error
"""
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

"""
Calculate PSNR
"""
def calc_psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))

def vgg_loss(sr, hr, vgg_model):
   sr_features = vgg_model((sr[:,:3,:,:]+1)/2.)
   hr_features = vgg_model((hr[:,:3,:,:]+1)/2.)
   vgg_loss = torch.mean((sr_features-hr_features)**2)
   return vgg_loss

"""
Train and val the model
"""
best_weights = copy.deepcopy(model.state_dict())
best_epoch = 0
best_psnr = 0.0
# make a directory called "weight_rcan" to store weights
os.makedirs('weight_rcan', exist_ok = True)
os.makedirs('result_images', exist_ok = True)
os.makedirs('results_test_visualization', exist_ok = True)

checkpoint_path = "checkpoint.pth"
# check to see if a checkpoint exists
if os.path.exists(checkpoint_path):
  checkpoint = torch.load(checkpoint_path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  optimizer_unet.load_state_dict(checkpoint['optimizer_unet_state_dict'])
  best_psnr = checkpoint['best_psnr']
  best_epoch = checkpoint['best_epoch']
  print(f"Checkpoint Loaded at epoch {best_epoch} and best psnr of {best_psnr}")
else:
    print("No Checkpoint Found")

num_epoch = 200
patience = 6 # number of epochs to wait without improvement
lr_decay_factor = 0.5
patience_counter = 0



for epoch in range(num_epoch):
    model.train()
    unet_model.train()
    vgg_model.eval()
    epoch_losses = AverageMeter()

    with tqdm(total=(len(train_dataset) - len(train_dataset) % batch_size)) as t:
        t.set_description('epoch: {}/{}'.format(epoch, num_epoch - 1))
        
        # training
        for data in train_dataloader:
            inputs, labels = data

            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            
            multi_scale_features = unet_model(inputs)

            preds = model(multi_scale_features)

            loss = 0.1 * criterion(preds, labels) + 0.0001 * vgg_loss(preds, labels, vgg_model) # combined loss

            epoch_losses.update(loss.item(), len(inputs))

            optimizer.zero_grad()
            optimizer_unet.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer_unet.step()

            t.set_postfix(loss='{:.6f}'.format(epoch_losses.avg))
            t.update(len(inputs))
        torch.save(model.state_dict(), os.path.join('weight_rcan', 'epoch_{}.pth'.format(epoch)))


        # validation
        model.eval()
        unet_model.eval()
        epoch_psnr = AverageMeter()

        for data in eval_dataloader:
            inputs, labels = data

            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            with torch.no_grad():
                multi_scale_features = unet_model(inputs)
                preds = model(multi_scale_features).clamp(0.0, 1.0)

            epoch_psnr.update(calc_psnr(preds, labels), len(inputs))
        current_psnr = epoch_psnr.avg
        print('eval psnr: {:.2f}'.format(current_psnr))
        if (epoch + 1) % 10 == 0:
          # Get a random image from the validation dataset
            random_index = random.randint(0, len(eval_dataset) - 1)
            lr_image, hr_image = eval_dataset[random_index]
            lr_image = torch.from_numpy(lr_image).to(device).unsqueeze(0)
            with torch.no_grad():
              multi_scale_features = unet_model(lr_image)
              enhanced_image = model(multi_scale_features).clamp(0,1)
              enhanced_image = enhanced_image.squeeze(0).permute(1, 2, 0).cpu().numpy()
              enhanced_image = np.asarray((enhanced_image[...,:3] )* 255, dtype=np.uint8) # take first 3 channels, and convert to uint8
            hr_image = np.transpose(hr_image, (1, 2, 0))
            hr_image = np.asarray(hr_image[...,:3] * 255, dtype = np.uint8) # take the first 3 channels, and make it uint8.
            hr_image = Image.fromarray(hr_image)
            enhanced_image = Image.fromarray(enhanced_image)
            hr_image.save(f'result_images/hr_epoch_{epoch + 1}.png')
            enhanced_image.save(f'result_images/enhanced_epoch_{epoch + 1}.png')

        if current_psnr > best_psnr:
            best_epoch = epoch
            best_psnr = current_psnr
            best_weights = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
          patience_counter += 1

        if patience_counter > patience:
            for param_group in optimizer.param_groups:
              param_group['lr'] *= lr_decay_factor
              print("reducing learning rate")
            for param_group in optimizer_unet.param_groups:
              param_group['lr'] *= lr_decay_factor
              print("reducing unet learning rate")
            patience_counter = 0
        # save checkpoint every epoch
        torch.save({
           'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
             'optimizer_unet_state_dict': optimizer_unet.state_dict(),
             'best_psnr': best_psnr,
             'best_epoch': best_epoch
        }, checkpoint_path)


    print('best epoch: {}, psnr: {:.2f}'.format(best_epoch, best_psnr))
    torch.save(best_weights, os.path.join('weight_rcan', 'best.pth'))
"""
Evaluate the model with test set
"""
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = RCAN(param).to(device)
unet_model = UNet(in_channels = 13, out_channels = 13, features = 64).to(device)
vgg_model = VGG19(pretrained = True, end_with = 'pool4').to(device)

state_dict = model.state_dict()
for n, p in torch.load('weight_rcan/best.pth', map_location=lambda storage, loc: storage).items():
    if n in state_dict.keys():
        state_dict[n].copy_(p)
    else:
        raise KeyError(n)

model.eval()
unet_model.eval()
from pytorch_ssim import ssim as pytorch_ssim_ssim
def ssim(img1, img2):
  window_size = 11
  (_, channel, _, _) = img1.size()
  window = create_window(window_size, channel)
  padding = int(window_size/2) # making sure that padding is an integer
  return pytorch_ssim_ssim(img1, img2, window, window_size, channel, padding).item()

# Adjust the test data path as needed, assuming you have a test folder with similar subfolders
test_dataset = CustomDataset(os.path.join(dataset_path, 'test'))
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1) # batch size is 1 here to evaluate each image independently


psnr_total = 0
ssim_total = 0
num_test_images = 0
for i, data in enumerate(test_dataloader):
    inputs, labels = data

    inputs = inputs.to(device, dtype=torch.float)
    labels = labels.to(device, dtype=torch.float)


    with torch.no_grad():
      multi_scale_features = unet_model(inputs)
      preds = model(multi_scale_features).clamp(0.0, 1.0)

    psnr = calc_psnr(labels, preds)
    psnr_total += psnr
    ssim_total += ssim(labels, preds) # corrected
    print('PSNR: {:.2f}'.format(psnr))
    print('SSIM: {:.2f}'.format(ssim_total))

    output = preds.mul_(255.0).clamp_(0.0, 255.0).squeeze(0).permute(1, 2, 0).byte().cpu().numpy()
    output = Image.fromarray(output)
    output.save(f'results_test_visualization/img_{i}.png')
    num_test_images += 1

psnr_total /= num_test_images
ssim_total /= num_test_images
print('PSNR_T: {:.4f}'.format(psnr_total))
print('SSIM_T: {:.4f}'.format(ssim_total))

Dataset folder already exists, skipping dataset creation...


C:\Users\CSE-312-02\AppData\Local\Temp\ipykernel_6628\2708574523.py:285: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Checkpoint Loaded at epoch 6 and best psnr of 38.449363708496094


epoch: 0/199: 100%|██████████| 18880/18880 [05:26<00:00, 60.30it/s, loss=0.000642]

eval psnr: 38.11


...

eval psnr: 38.56


epoch: 199/199: 100%|██████████| 18880/18880 [05:36<00:00, 56.08it/s, loss=0.000444]


best epoch: 42, psnr: 38.61


C:\Users\CSE-312-02\AppData\Local\Temp\ipykernel_6628\2708574523.py:411: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  for n, p in torch.load('weight_rcan/best.pth', map_loc

ModuleNotFoundError: No module named 'pytorch_ssim'